## 1. About

This noteboook takes in as input the data gotten from the [SPICE](https://en.wikipedia.org/wiki/SPICE) simulation about the circuit and reformats them into a format more usable.

### 1.1 About this New Version

The old notebook was laggy and not well documented, this notebook aims to improve this. 

Mainly by exchanging the **pandas** libary through **polars** libary.

### 1.2 Explanation of the Data

### 1.3 General

Learnable SG Data

#### Dataset 1

Inputs:  
1. Processed_VIN_of_Constant_1st_Neuron_Vin0-1.csv
2. Processed_VIN_of_Constant_1st_Neuron_Vin1-2.csv

Outputs:  
1. Processed_output_Vout_of_Constant_1st_Neuron_Vin0-1.csv
2. Processed_output_Vout_of_Constant_1st_Neuron_Vin1-2.csv
3. Processed_output_Vout_of_Constant_2nd_Neuron_Vin0-1.csv
4. Processed_output_Vout_of_Constant_2nd_Neuron_Vin1-2.csv

Parameters:     
- a. C1 (10u-100u)
- b. CIN (10u-100u)
- c. COUT (10u-100u)
- d. L (40u-100u)
- e. R1 (20k-50k)
- f. W1 (600u-1000u)
- g. VIN (input voltage) (0:1, 1:2)

#### Dataset 2

Inputs:  
1. Processed_Vin_sin1.csv
2. Processed_Vin_sin2.csv
3. Processed_Vin_sin3.csv

Outputs:  
1. Processed_output_Vout_of_Sin1.csv
2. Processed_output_Vout_of_Sin2.csv
3. Processed_output_Vout_of_Sin3.csv

Parameters:    
- a. C1 (10u-100u)
- b. CIN (10u-100u)
- c. COUT (10u-100u)
- d. L (40u-100u)
- e. R1 (20k-50k)
- f. W1 (600u-1000u)
- g. a  (inputs to create sin wave of multiple freq) (0:0.25, 0.25:0.5,0.5:1)
- h. b (0:180,180:360,180:360)
- i. f (1:3,3:5,3:5)
  

In [1]:
# Number of Entries of the dataset we want to use
# For Testing purpose to generate all the data set to "None"
NUMBER_DATASET_LENGTH = None

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import re
import warnings
import polars as pl

## 2. Processing Inputs of Dataset 1

We have input voltage data (VIN) in df_input_1 and df_input_2.
And output voltage data (VOUT) in df_output_2 and df_output_2.

Each CSV is organized like this:

```csv
X (time?)	Column 1	Column 2	Column 3	…
0.00	0.04193	0.04193	0.04193	…
0.01	0.04193	0.04193	0.04193	…
```

Columns are named with parameter information inside parentheses, e.g.:

```
/Vout_buffer2 (C1=9.4e-05,CIN=7e-05,COUT=3.5e-05,L=6.6e-05,R1=20473,VIN=1.111111,w1=0.000776)
```

Each column corresponds to a simulation run with a particular set of circuit parameters and input voltage and each row entry represents one timestep, whereby the entry the current input voltage of the time step is.

In [3]:
DATA_LOCATION = "data/V2/"

In [4]:
df_input_1 = pl.read_csv(str(DATA_LOCATION + "Input/" + "Processed_VIN_of_Constant_1st_Neuron_Vin0-1_v2.csv"))
df_input_2 = pl.read_csv(str(DATA_LOCATION + "Input/" + "Processed_VIN_of_Constant_1st_Neuron_Vin1-2_v2.csv"))
df_input_1.head()

FileNotFoundError: No such file or directory (os error 2): data/V2/Input/Processed_VIN_of_Constant_1st_Neuron_Vin0-1_v2.csv

In [5]:
df_output_1 = pl.read_csv(str(DATA_LOCATION + "/Output/" + "Processed_output_Vout_of_Constant_1st_Neuron_Vin0-1_v2.csv"))
df_output_2 = pl.read_csv(str(DATA_LOCATION + "/Output/" + "Processed_output_Vout_of_Constant_1st_Neuron_Vin1-2_v2.csv"))
df_output_2.head()

X,"/Vout_buffer2 (C1=9.4e-05,CIN=7e-05,COUT=3.5e-05,L=6.6e-05,R1=20473,VIN=1.111111,w1=0.000776) Y","/Vout_buffer2 (C1=9.4e-05,CIN=7e-05,COUT=3.5e-05,L=6.6e-05,R1=20473,VIN=1.222222,w1=0.000776) Y","/Vout_buffer2 (C1=9.4e-05,CIN=7e-05,COUT=3.5e-05,L=6.6e-05,R1=20473,VIN=1.333333,w1=0.000776) Y","/Vout_buffer2 (C1=9.4e-05,CIN=7e-05,COUT=3.5e-05,L=6.6e-05,R1=20473,VIN=1.444444,w1=0.000776) Y","/Vout_buffer2 (C1=9.4e-05,CIN=7e-05,COUT=3.5e-05,L=6.6e-05,R1=20473,VIN=1.555556,w1=0.000776) Y","/Vout_buffer2 (C1=9.4e-05,CIN=7e-05,COUT=3.5e-05,L=6.6e-05,R1=20473,VIN=1.666667,w1=0.000776) Y","/Vout_buffer2 (C1=9.4e-05,CIN=7e-05,COUT=3.5e-05,L=6.6e-05,R1=20473,VIN=1.777778,w1=0.000776) Y","/Vout_buffer2 (C1=9.4e-05,CIN=7e-05,COUT=3.5e-05,L=6.6e-05,R1=20473,VIN=1.888889,w1=0.000776) Y","/Vout_buffer2 (C1=9.4e-05,CIN=7e-05,COUT=3.5e-05,L=6.6e-05,R1=20473,VIN=2,w1=0.000776) Y","/Vout_buffer2 (C1=3.8e-05,CIN=7.7e-05,COUT=6.6e-05,L=7.8e-05,R1=22511,VIN=1,w1=0.000734) Y","/Vout_buffer2 (C1=3.8e-05,CIN=7.7e-05,COUT=6.6e-05,L=7.8e-05,R1=22511,VIN=1.111111,w1=0.000734) Y","/Vout_buffer2 (C1=3.8e-05,CIN=7.7e-05,COUT=6.6e-05,L=7.8e-05,R1=22511,VIN=1.222222,w1=0.000734) Y","/Vout_buffer2 (C1=3.8e-05,CIN=7.7e-05,COUT=6.6e-05,L=7.8e-05,R1=22511,VIN=1.333333,w1=0.000734) Y","/Vout_buffer2 (C1=3.8e-05,CIN=7.7e-05,COUT=6.6e-05,L=7.8e-05,R1=22511,VIN=1.444444,w1=0.000734) Y","/Vout_buffer2 (C1=3.8e-05,CIN=7.7e-05,COUT=6.6e-05,L=7.8e-05,R1=22511,VIN=1.555556,w1=0.000734) Y","/Vout_buffer2 (C1=3.8e-05,CIN=7.7e-05,COUT=6.6e-05,L=7.8e-05,R1=22511,VIN=1.666667,w1=0.000734) Y","/Vout_buffer2 (C1=3.8e-05,CIN=7.7e-05,COUT=6.6e-05,L=7.8e-05,R1=22511,VIN=1.777778,w1=0.000734) Y","/Vout_buffer2 (C1=3.8e-05,CIN=7.7e-05,COUT=6.6e-05,L=7.8e-05,R1=22511,VIN=1.888889,w1=0.000734) Y","/Vout_buffer2 (C1=3.8e-05,CIN=7.7e-05,COUT=6.6e-05,L=7.8e-05,R1=22511,VIN=2,w1=0.000734) Y","/Vout_buffer2 (C1=8.4e-05,CIN=5.8e-05,COUT=5.8e-05,L=5.5e-05,R1=37296,VIN=1,w1=0.00065) Y","/Vout_buffer2 (C1=8.4e-05,CIN=5.8e-05,COUT=5.8e-05,L=5.5e-05,R1=37296,VIN=1.111111,w1=0.00065) Y","/Vout_buffer2 (C1=8.4e-05,CIN=5.8e-05,COUT=5.8e-05,L=5.5e-05,R1=37296,VIN=1.222222,w1=0.00065) Y","/Vout_buffer2 (C1=8.4e-05,CIN=5.8e-05,COUT=5.8e-05,L=5.5e-05,R1=37296,VIN=1.333333,w1=0.00065) Y","/Vout_buffer2 (C1=8.4e-05,CIN=5.8e-05,COUT=5.8e-05,L=5.5e-05,R1=37296,VIN=1.444444,w1=0.00065) Y","/Vout_buffer2 (C1=8.4e-05,CIN=5.8e-05,COUT=5.8e-05,L=5.5e-05,R1=37296,VIN=1.555556,w1=0.00065) Y","/Vout_buffer2 (C1=8.4e-05,CIN=5.8e-05,COUT=5.8e-05,L=5.5e-05,R1=37296,VIN=1.666667,w1=0.00065) Y","/Vout_buffer2 (C1=8.4e-05,CIN=5.8e-05,COUT=5.8e-05,L=5.5e-05,R1=37296,VIN=1.777778,w1=0.00065) Y","/Vout_buffer2 (C1=8.4e-05,CIN=5.8e-05,COUT=5.8e-05,L=5.5e-05,R1=37296,VIN=1.888889,w1=0.00065) Y","/Vout_buffer2 (C1=8.4e-05,CIN=5.8e-05,COUT=5.8e-05,L=5.5e-05,R1=37296,VIN=2,w1=0.00065) Y","/Vout_buffer2 (C1=5.1e-05,CIN=4.9e-05,COUT=4.7e-05,L=6.4e-05,R1=20139,VIN=1,w1=0.000999) Y","/Vout_buffer2 (C1=5.1e-05,CIN=4.9e-05,COUT=4.7e-05,L=6.4e-05,R1=20139,VIN=1.111111,w1=0.000999) Y","/Vout_buffer2 (C1=5.1e-05,CIN=4.9e-05,COUT=4.7e-05,L=6.4e-05,R1=20139,VIN=1.222222,w1=0.000999) Y","/Vout_buffer2 (C1=5.1e-05,CIN=4.9e-05,COUT=4.7e-05,L=6.4e-05,R1=20139,VIN=1.333333,w1=0.000999) Y","/Vout_buffer2 (C1=5.1e-05,CIN=4.9e-05,COUT=4.7e-05,L=6.4e-05,R1=20139,VIN=1.444444,w1=0.000999) Y","/Vout_buffer2 (C1=5.1e-05,CIN=4.9e-05,COUT=4.7e-05,L=6.4e-05,R1=20139,VIN=1.555556,w1=0.000999) Y","/Vout_buffer2 (C1=5.1e-05,CIN=4.9e-05,COUT=4.7e-05,L=6.4e-05,R1=20139,VIN=1.666667,w1=0.000999) Y",…,"/Vout_buffer2 (C1=5.1e-05,CIN=7.5e-05,COUT=9.8e-05,L=5.6e-05,R1=30871,VIN=1.333333,w1=0.000918) Y","/Vout_buffer2 (C1=5.1e-05,CIN=7.5e-05,COUT=9.8e-05,L=5.6e-05,R1=30871,VIN=1.444444,w1=0.000918) Y","/Vout_buffer2 (C1=5.1e-05,CIN=7.5e-05,COUT=9.8e-05,L=5.6e-05,R1=30871,VIN=1.555556,w1=0.000918) Y","/Vout_buffer2 (C1=5.1e-05,CIN=7.5e-05,COUT=9.8e-05,L=5.6e-05,R1=30871,VIN=1.666667,w1=0.000918) Y","/Vout_buffer2 (C1=5.1e-05,CIN=7.5e-05,COUT=9.8e-05,L=5.6e-05,R1=3

### 2.1 Goal of Expand Function

The goal with this function is to make is so that the paramaeters of the simulation run that are embedded in the column name are instead added to the tail of the dataframe.
This means afetr using the expand fucntion, each column in the dataframe is one of the simulation run, where each entry is the voltage at a particular time, while at the end of the column the parameetrs are added, for eahc parameetr one new row. 

In [6]:
def expand(df, params=None, id_col="X"):
    if params is None:
        params = ["C1", "CIN", "COUT", "L", "R1", "w1"]

    cols = [c for c in df.columns if c != id_col]

    def parse_col(colname: str):
        try:
            inside = colname.split("(", 1)[1].split(")", 1)[0]
        except Exception:
            return {}
        parsed = {}
        for part in inside.split(","):
            if "=" not in part:
                continue
            k, v = part.split("=", 1)
            try:
                parsed[k.strip()] = float(v.strip())
            except Exception:
                parsed[k.strip()] = None
        return parsed

    parsed = [parse_col(c) for c in cols]

    rows = []
    for p in params:
        row = {id_col: p}
        for i, c in enumerate(cols):
            row[c] = parsed[i].get(p)
        rows.append(row)
    params_df = pl.DataFrame(rows)

    # concat with relaxed typing
    combined = pl.concat([df.with_columns(pl.col(id_col).cast(pl.Utf8)), params_df],
                         how="vertical_relaxed")

    # overwrite the id_col with sequential labels like in pandas
    labels = [str(i) for i in range(len(df))] + params  # 0..N-1, then C1, ...
    combined = combined.with_columns(pl.Series(id_col, labels))

    return combined

In [7]:
%%time

df_input_1_combined = expand(df_input_1[:NUMBER_DATASET_LENGTH])
df_input_2_combined = expand(df_input_2[:NUMBER_DATASET_LENGTH])

CPU times: user 8.74 s, sys: 1.28 s, total: 10 s
Wall time: 6.83 s


In [8]:
df_input_1_combined.head()

X,"/VIN (C1=9.4e-05,CIN=7e-05,COUT=3.5e-05,L=6.6e-05,R1=20473,VIN=0,w1=0.000776) Y","/VIN (C1=9.4e-05,CIN=7e-05,COUT=3.5e-05,L=6.6e-05,R1=20473,VIN=0.2222222,w1=0.000776) Y","/VIN (C1=9.4e-05,CIN=7e-05,COUT=3.5e-05,L=6.6e-05,R1=20473,VIN=0.3333333,w1=0.000776) Y","/VIN (C1=9.4e-05,CIN=7e-05,COUT=3.5e-05,L=6.6e-05,R1=20473,VIN=0.4444444,w1=0.000776) Y","/VIN (C1=9.4e-05,CIN=7e-05,COUT=3.5e-05,L=6.6e-05,R1=20473,VIN=0.5555556,w1=0.000776) Y","/VIN (C1=9.4e-05,CIN=7e-05,COUT=3.5e-05,L=6.6e-05,R1=20473,VIN=0.6666667,w1=0.000776) Y","/VIN (C1=9.4e-05,CIN=7e-05,COUT=3.5e-05,L=6.6e-05,R1=20473,VIN=0.7777778,w1=0.000776) Y","/VIN (C1=9.4e-05,CIN=7e-05,COUT=3.5e-05,L=6.6e-05,R1=20473,VIN=0.8888889,w1=0.000776) Y","/VIN (C1=9.4e-05,CIN=7e-05,COUT=3.5e-05,L=6.6e-05,R1=20473,VIN=1,w1=0.000776) Y","/VIN (C1=3.8e-05,CIN=7.7e-05,COUT=6.6e-05,L=7.8e-05,R1=22511,VIN=0,w1=0.000734) Y","/VIN (C1=3.8e-05,CIN=7.7e-05,COUT=6.6e-05,L=7.8e-05,R1=22511,VIN=0.1111111,w1=0.000734) Y","/VIN (C1=3.8e-05,CIN=7.7e-05,COUT=6.6e-05,L=7.8e-05,R1=22511,VIN=0.2222222,w1=0.000734) Y","/VIN (C1=3.8e-05,CIN=7.7e-05,COUT=6.6e-05,L=7.8e-05,R1=22511,VIN=0.3333333,w1=0.000734) Y","/VIN (C1=3.8e-05,CIN=7.7e-05,COUT=6.6e-05,L=7.8e-05,R1=22511,VIN=0.4444444,w1=0.000734) Y","/VIN (C1=3.8e-05,CIN=7.7e-05,COUT=6.6e-05,L=7.8e-05,R1=22511,VIN=0.5555556,w1=0.000734) Y","/VIN (C1=3.8e-05,CIN=7.7e-05,COUT=6.6e-05,L=7.8e-05,R1=22511,VIN=0.6666667,w1=0.000734) Y","/VIN (C1=3.8e-05,CIN=7.7e-05,COUT=6.6e-05,L=7.8e-05,R1=22511,VIN=0.7777778,w1=0.000734) Y","/VIN (C1=3.8e-05,CIN=7.7e-05,COUT=6.6e-05,L=7.8e-05,R1=22511,VIN=0.8888889,w1=0.000734) Y","/VIN (C1=3.8e-05,CIN=7.7e-05,COUT=6.6e-05,L=7.8e-05,R1=22511,VIN=1,w1=0.000734) Y","/VIN (C1=8.4e-05,CIN=5.8e-05,COUT=5.8e-05,L=5.5e-05,R1=37296,VIN=0,w1=0.00065) Y","/VIN (C1=8.4e-05,CIN=5.8e-05,COUT=5.8e-05,L=5.5e-05,R1=37296,VIN=0.1111111,w1=0.00065) Y","/VIN (C1=8.4e-05,CIN=5.8e-05,COUT=5.8e-05,L=5.5e-05,R1=37296,VIN=0.2222222,w1=0.00065) Y","/VIN (C1=8.4e-05,CIN=5.8e-05,COUT=5.8e-05,L=5.5e-05,R1=37296,VIN=0.3333333,w1=0.00065) Y","/VIN (C1=8.4e-05,CIN=5.8e-05,COUT=5.8e-05,L=5.5e-05,R1=37296,VIN=0.4444444,w1=0.00065) Y","/VIN (C1=8.4e-05,CIN=5.8e-05,COUT=5.8e-05,L=5.5e-05,R1=37296,VIN=0.5555556,w1=0.00065) Y","/VIN (C1=8.4e-05,CIN=5.8e-05,COUT=5.8e-05,L=5.5e-05,R1=37296,VIN=0.6666667,w1=0.00065) Y","/VIN (C1=8.4e-05,CIN=5.8e-05,COUT=5.8e-05,L=5.5e-05,R1=37296,VIN=0.7777778,w1=0.00065) Y","/VIN (C1=8.4e-05,CIN=5.8e-05,COUT=5.8e-05,L=5.5e-05,R1=37296,VIN=0.8888889,w1=0.00065) Y","/VIN (C1=8.4e-05,CIN=5.8e-05,COUT=5.8e-05,L=5.5e-05,R1=37296,VIN=1,w1=0.00065) Y","/VIN (C1=5.1e-05,CIN=4.9e-05,COUT=4.7e-05,L=6.4e-05,R1=20139,VIN=0,w1=0.000999) Y","/VIN (C1=5.1e-05,CIN=4.9e-05,COUT=4.7e-05,L=6.4e-05,R1=20139,VIN=0.1111111,w1=0.000999) Y","/VIN (C1=5.1e-05,CIN=4.9e-05,COUT=4.7e-05,L=6.4e-05,R1=20139,VIN=0.2222222,w1=0.000999) Y","/VIN (C1=5.1e-05,CIN=4.9e-05,COUT=4.7e-05,L=6.4e-05,R1=20139,VIN=0.3333333,w1=0.000999) Y","/VIN (C1=5.1e-05,CIN=4.9e-05,COUT=4.7e-05,L=6.4e-05,R1=20139,VIN=0.4444444,w1=0.000999) Y","/VIN (C1=5.1e-05,CIN=4.9e-05,COUT=4.7e-05,L=6.4e-05,R1=20139,VIN=0.5555556,w1=0.000999) Y","/VIN (C1=5.1e-05,CIN=4.9e-05,COUT=4.7e-05,L=6.4e-05,R1=20139,VIN=0.6666667,w1=0.000999) Y",…,"/VIN (C1=5.1e-05,CIN=7.5e-05,COUT=9.8e-05,L=5.6e-05,R1=30871,VIN=0.3333333,w1=0.000918) Y","/VIN (C1=5.1e-05,CIN=7.5e-05,COUT=9.8e-05,L=5.6e-05,R1=30871,VIN=0.4444444,w1=0.000918) Y","/VIN (C1=5.1e-05,CIN=7.5e-05,COUT=9.8e-05,L=5.6e-05,R1=30871,VIN=0.5555556,w1=0.000918) Y","/VIN (C1=5.1e-05,CIN=7.5e-05,COUT=9.8e-05,L=5.6e-05,R1=30871,VIN=0.6666667,w1=0.000918) Y","/VIN (C1=5.1e-05,CIN=7.5e-05,COUT=9.8e-05,L=5.6e-05,R1=30871,VIN=0.7777778,w1=0.000918) Y","/VIN (C1=5.1e-05,CIN=7.5e-05,COUT=9.8e-05,L=5.6e-05,R1=30871,VIN=0.8888889,w1=0.000918) Y","/VIN (C1=5.1e-05,CIN=7.5e-05,COUT=9.8e-05,L=5.6e-05,R1=30871,VIN=1,w1=0.000918) Y","/VIN (C1=5.7e-05,CIN=8.9e-05,COUT=1.9e-05,L=4.3e-05,R1=46163,VIN=0,w1=0.000772) Y","/VIN (C1=5.7e-05,CIN=8.9e-05,COUT=1.9e-05,L=4.3

In [9]:
df_input_1_combined.tail()

X,"/VIN (C1=9.4e-05,CIN=7e-05,COUT=3.5e-05,L=6.6e-05,R1=20473,VIN=0,w1=0.000776) Y","/VIN (C1=9.4e-05,CIN=7e-05,COUT=3.5e-05,L=6.6e-05,R1=20473,VIN=0.2222222,w1=0.000776) Y","/VIN (C1=9.4e-05,CIN=7e-05,COUT=3.5e-05,L=6.6e-05,R1=20473,VIN=0.3333333,w1=0.000776) Y","/VIN (C1=9.4e-05,CIN=7e-05,COUT=3.5e-05,L=6.6e-05,R1=20473,VIN=0.4444444,w1=0.000776) Y","/VIN (C1=9.4e-05,CIN=7e-05,COUT=3.5e-05,L=6.6e-05,R1=20473,VIN=0.5555556,w1=0.000776) Y","/VIN (C1=9.4e-05,CIN=7e-05,COUT=3.5e-05,L=6.6e-05,R1=20473,VIN=0.6666667,w1=0.000776) Y","/VIN (C1=9.4e-05,CIN=7e-05,COUT=3.5e-05,L=6.6e-05,R1=20473,VIN=0.7777778,w1=0.000776) Y","/VIN (C1=9.4e-05,CIN=7e-05,COUT=3.5e-05,L=6.6e-05,R1=20473,VIN=0.8888889,w1=0.000776) Y","/VIN (C1=9.4e-05,CIN=7e-05,COUT=3.5e-05,L=6.6e-05,R1=20473,VIN=1,w1=0.000776) Y","/VIN (C1=3.8e-05,CIN=7.7e-05,COUT=6.6e-05,L=7.8e-05,R1=22511,VIN=0,w1=0.000734) Y","/VIN (C1=3.8e-05,CIN=7.7e-05,COUT=6.6e-05,L=7.8e-05,R1=22511,VIN=0.1111111,w1=0.000734) Y","/VIN (C1=3.8e-05,CIN=7.7e-05,COUT=6.6e-05,L=7.8e-05,R1=22511,VIN=0.2222222,w1=0.000734) Y","/VIN (C1=3.8e-05,CIN=7.7e-05,COUT=6.6e-05,L=7.8e-05,R1=22511,VIN=0.3333333,w1=0.000734) Y","/VIN (C1=3.8e-05,CIN=7.7e-05,COUT=6.6e-05,L=7.8e-05,R1=22511,VIN=0.4444444,w1=0.000734) Y","/VIN (C1=3.8e-05,CIN=7.7e-05,COUT=6.6e-05,L=7.8e-05,R1=22511,VIN=0.5555556,w1=0.000734) Y","/VIN (C1=3.8e-05,CIN=7.7e-05,COUT=6.6e-05,L=7.8e-05,R1=22511,VIN=0.6666667,w1=0.000734) Y","/VIN (C1=3.8e-05,CIN=7.7e-05,COUT=6.6e-05,L=7.8e-05,R1=22511,VIN=0.7777778,w1=0.000734) Y","/VIN (C1=3.8e-05,CIN=7.7e-05,COUT=6.6e-05,L=7.8e-05,R1=22511,VIN=0.8888889,w1=0.000734) Y","/VIN (C1=3.8e-05,CIN=7.7e-05,COUT=6.6e-05,L=7.8e-05,R1=22511,VIN=1,w1=0.000734) Y","/VIN (C1=8.4e-05,CIN=5.8e-05,COUT=5.8e-05,L=5.5e-05,R1=37296,VIN=0,w1=0.00065) Y","/VIN (C1=8.4e-05,CIN=5.8e-05,COUT=5.8e-05,L=5.5e-05,R1=37296,VIN=0.1111111,w1=0.00065) Y","/VIN (C1=8.4e-05,CIN=5.8e-05,COUT=5.8e-05,L=5.5e-05,R1=37296,VIN=0.2222222,w1=0.00065) Y","/VIN (C1=8.4e-05,CIN=5.8e-05,COUT=5.8e-05,L=5.5e-05,R1=37296,VIN=0.3333333,w1=0.00065) Y","/VIN (C1=8.4e-05,CIN=5.8e-05,COUT=5.8e-05,L=5.5e-05,R1=37296,VIN=0.4444444,w1=0.00065) Y","/VIN (C1=8.4e-05,CIN=5.8e-05,COUT=5.8e-05,L=5.5e-05,R1=37296,VIN=0.5555556,w1=0.00065) Y","/VIN (C1=8.4e-05,CIN=5.8e-05,COUT=5.8e-05,L=5.5e-05,R1=37296,VIN=0.6666667,w1=0.00065) Y","/VIN (C1=8.4e-05,CIN=5.8e-05,COUT=5.8e-05,L=5.5e-05,R1=37296,VIN=0.7777778,w1=0.00065) Y","/VIN (C1=8.4e-05,CIN=5.8e-05,COUT=5.8e-05,L=5.5e-05,R1=37296,VIN=0.8888889,w1=0.00065) Y","/VIN (C1=8.4e-05,CIN=5.8e-05,COUT=5.8e-05,L=5.5e-05,R1=37296,VIN=1,w1=0.00065) Y","/VIN (C1=5.1e-05,CIN=4.9e-05,COUT=4.7e-05,L=6.4e-05,R1=20139,VIN=0,w1=0.000999) Y","/VIN (C1=5.1e-05,CIN=4.9e-05,COUT=4.7e-05,L=6.4e-05,R1=20139,VIN=0.1111111,w1=0.000999) Y","/VIN (C1=5.1e-05,CIN=4.9e-05,COUT=4.7e-05,L=6.4e-05,R1=20139,VIN=0.2222222,w1=0.000999) Y","/VIN (C1=5.1e-05,CIN=4.9e-05,COUT=4.7e-05,L=6.4e-05,R1=20139,VIN=0.3333333,w1=0.000999) Y","/VIN (C1=5.1e-05,CIN=4.9e-05,COUT=4.7e-05,L=6.4e-05,R1=20139,VIN=0.4444444,w1=0.000999) Y","/VIN (C1=5.1e-05,CIN=4.9e-05,COUT=4.7e-05,L=6.4e-05,R1=20139,VIN=0.5555556,w1=0.000999) Y","/VIN (C1=5.1e-05,CIN=4.9e-05,COUT=4.7e-05,L=6.4e-05,R1=20139,VIN=0.6666667,w1=0.000999) Y",…,"/VIN (C1=5.1e-05,CIN=7.5e-05,COUT=9.8e-05,L=5.6e-05,R1=30871,VIN=0.3333333,w1=0.000918) Y","/VIN (C1=5.1e-05,CIN=7.5e-05,COUT=9.8e-05,L=5.6e-05,R1=30871,VIN=0.4444444,w1=0.000918) Y","/VIN (C1=5.1e-05,CIN=7.5e-05,COUT=9.8e-05,L=5.6e-05,R1=30871,VIN=0.5555556,w1=0.000918) Y","/VIN (C1=5.1e-05,CIN=7.5e-05,COUT=9.8e-05,L=5.6e-05,R1=30871,VIN=0.6666667,w1=0.000918) Y","/VIN (C1=5.1e-05,CIN=7.5e-05,COUT=9.8e-05,L=5.6e-05,R1=30871,VIN=0.7777778,w1=0.000918) Y","/VIN (C1=5.1e-05,CIN=7.5e-05,COUT=9.8e-05,L=5.6e-05,R1=30871,VIN=0.8888889,w1=0.000918) Y","/VIN (C1=5.1e-05,CIN=7.5e-05,COUT=9.8e-05,L=5.6e-05,R1=30871,VIN=1,w1=0.000918) Y","/VIN (C1=5.7e-05,CIN=8.9e-05,COUT=1.9e-05,L=4.3e-05,R1=46163,VIN=0,w1=0.000772) Y","/VIN (C1=5.7e-05,CIN=8.9e-05,COUT=1.9e-05,L=4.3

Then concatenate them side by side, so your final input DataFrame dfin_const contains:
- All signal traces from dfin1 and dfin2
- All the extracted circuit parameters repeated for each corresponding trace  

In [10]:
def safe_concat(dfs, how="horizontal"):
    """
    Concatenate Polars DataFrames horizontally,
    automatically renaming duplicate columns (X, X_2, X_3, ...).
    """
    seen = {}
    renamed = []
    
    for df in dfs:
        new_cols = []
        for c in df.columns:
            if c not in seen:
                seen[c] = 1
                new_cols.append(c)
            else:
                seen[c] += 1
                new_cols.append(f"{c}_{seen[c]}")
        renamed.append(df.rename(dict(zip(df.columns, new_cols))))
    
    return pl.concat(renamed, how=how)

In [11]:
%%time
df_input_dataset_1_concat = safe_concat([df_input_1_combined, df_input_2_combined])

df_input_dataset_1_concat.tail(8)

CPU times: user 289 ms, sys: 30.8 ms, total: 319 ms
Wall time: 291 ms


X,"/VIN (C1=9.4e-05,CIN=7e-05,COUT=3.5e-05,L=6.6e-05,R1=20473,VIN=0,w1=0.000776) Y","/VIN (C1=9.4e-05,CIN=7e-05,COUT=3.5e-05,L=6.6e-05,R1=20473,VIN=0.2222222,w1=0.000776) Y","/VIN (C1=9.4e-05,CIN=7e-05,COUT=3.5e-05,L=6.6e-05,R1=20473,VIN=0.3333333,w1=0.000776) Y","/VIN (C1=9.4e-05,CIN=7e-05,COUT=3.5e-05,L=6.6e-05,R1=20473,VIN=0.4444444,w1=0.000776) Y","/VIN (C1=9.4e-05,CIN=7e-05,COUT=3.5e-05,L=6.6e-05,R1=20473,VIN=0.5555556,w1=0.000776) Y","/VIN (C1=9.4e-05,CIN=7e-05,COUT=3.5e-05,L=6.6e-05,R1=20473,VIN=0.6666667,w1=0.000776) Y","/VIN (C1=9.4e-05,CIN=7e-05,COUT=3.5e-05,L=6.6e-05,R1=20473,VIN=0.7777778,w1=0.000776) Y","/VIN (C1=9.4e-05,CIN=7e-05,COUT=3.5e-05,L=6.6e-05,R1=20473,VIN=0.8888889,w1=0.000776) Y","/VIN (C1=9.4e-05,CIN=7e-05,COUT=3.5e-05,L=6.6e-05,R1=20473,VIN=1,w1=0.000776) Y","/VIN (C1=3.8e-05,CIN=7.7e-05,COUT=6.6e-05,L=7.8e-05,R1=22511,VIN=0,w1=0.000734) Y","/VIN (C1=3.8e-05,CIN=7.7e-05,COUT=6.6e-05,L=7.8e-05,R1=22511,VIN=0.1111111,w1=0.000734) Y","/VIN (C1=3.8e-05,CIN=7.7e-05,COUT=6.6e-05,L=7.8e-05,R1=22511,VIN=0.2222222,w1=0.000734) Y","/VIN (C1=3.8e-05,CIN=7.7e-05,COUT=6.6e-05,L=7.8e-05,R1=22511,VIN=0.3333333,w1=0.000734) Y","/VIN (C1=3.8e-05,CIN=7.7e-05,COUT=6.6e-05,L=7.8e-05,R1=22511,VIN=0.4444444,w1=0.000734) Y","/VIN (C1=3.8e-05,CIN=7.7e-05,COUT=6.6e-05,L=7.8e-05,R1=22511,VIN=0.5555556,w1=0.000734) Y","/VIN (C1=3.8e-05,CIN=7.7e-05,COUT=6.6e-05,L=7.8e-05,R1=22511,VIN=0.6666667,w1=0.000734) Y","/VIN (C1=3.8e-05,CIN=7.7e-05,COUT=6.6e-05,L=7.8e-05,R1=22511,VIN=0.7777778,w1=0.000734) Y","/VIN (C1=3.8e-05,CIN=7.7e-05,COUT=6.6e-05,L=7.8e-05,R1=22511,VIN=0.8888889,w1=0.000734) Y","/VIN (C1=3.8e-05,CIN=7.7e-05,COUT=6.6e-05,L=7.8e-05,R1=22511,VIN=1,w1=0.000734) Y","/VIN (C1=8.4e-05,CIN=5.8e-05,COUT=5.8e-05,L=5.5e-05,R1=37296,VIN=0,w1=0.00065) Y","/VIN (C1=8.4e-05,CIN=5.8e-05,COUT=5.8e-05,L=5.5e-05,R1=37296,VIN=0.1111111,w1=0.00065) Y","/VIN (C1=8.4e-05,CIN=5.8e-05,COUT=5.8e-05,L=5.5e-05,R1=37296,VIN=0.2222222,w1=0.00065) Y","/VIN (C1=8.4e-05,CIN=5.8e-05,COUT=5.8e-05,L=5.5e-05,R1=37296,VIN=0.3333333,w1=0.00065) Y","/VIN (C1=8.4e-05,CIN=5.8e-05,COUT=5.8e-05,L=5.5e-05,R1=37296,VIN=0.4444444,w1=0.00065) Y","/VIN (C1=8.4e-05,CIN=5.8e-05,COUT=5.8e-05,L=5.5e-05,R1=37296,VIN=0.5555556,w1=0.00065) Y","/VIN (C1=8.4e-05,CIN=5.8e-05,COUT=5.8e-05,L=5.5e-05,R1=37296,VIN=0.6666667,w1=0.00065) Y","/VIN (C1=8.4e-05,CIN=5.8e-05,COUT=5.8e-05,L=5.5e-05,R1=37296,VIN=0.7777778,w1=0.00065) Y","/VIN (C1=8.4e-05,CIN=5.8e-05,COUT=5.8e-05,L=5.5e-05,R1=37296,VIN=0.8888889,w1=0.00065) Y","/VIN (C1=8.4e-05,CIN=5.8e-05,COUT=5.8e-05,L=5.5e-05,R1=37296,VIN=1,w1=0.00065) Y","/VIN (C1=5.1e-05,CIN=4.9e-05,COUT=4.7e-05,L=6.4e-05,R1=20139,VIN=0,w1=0.000999) Y","/VIN (C1=5.1e-05,CIN=4.9e-05,COUT=4.7e-05,L=6.4e-05,R1=20139,VIN=0.1111111,w1=0.000999) Y","/VIN (C1=5.1e-05,CIN=4.9e-05,COUT=4.7e-05,L=6.4e-05,R1=20139,VIN=0.2222222,w1=0.000999) Y","/VIN (C1=5.1e-05,CIN=4.9e-05,COUT=4.7e-05,L=6.4e-05,R1=20139,VIN=0.3333333,w1=0.000999) Y","/VIN (C1=5.1e-05,CIN=4.9e-05,COUT=4.7e-05,L=6.4e-05,R1=20139,VIN=0.4444444,w1=0.000999) Y","/VIN (C1=5.1e-05,CIN=4.9e-05,COUT=4.7e-05,L=6.4e-05,R1=20139,VIN=0.5555556,w1=0.000999) Y","/VIN (C1=5.1e-05,CIN=4.9e-05,COUT=4.7e-05,L=6.4e-05,R1=20139,VIN=0.6666667,w1=0.000999) Y",…,"/VIN (C1=5.1e-05,CIN=7.5e-05,COUT=9.8e-05,L=5.6e-05,R1=30871,VIN=1.333333,w1=0.000918) Y","/VIN (C1=5.1e-05,CIN=7.5e-05,COUT=9.8e-05,L=5.6e-05,R1=30871,VIN=1.444444,w1=0.000918) Y","/VIN (C1=5.1e-05,CIN=7.5e-05,COUT=9.8e-05,L=5.6e-05,R1=30871,VIN=1.555556,w1=0.000918) Y","/VIN (C1=5.1e-05,CIN=7.5e-05,COUT=9.8e-05,L=5.6e-05,R1=30871,VIN=1.666667,w1=0.000918) Y","/VIN (C1=5.1e-05,CIN=7.5e-05,COUT=9.8e-05,L=5.6e-05,R1=30871,VIN=1.777778,w1=0.000918) Y","/VIN (C1=5.1e-05,CIN=7.5e-05,COUT=9.8e-05,L=5.6e-05,R1=30871,VIN=1.888889,w1=0.000918) Y","/VIN (C1=5.1e-05,CIN=7.5e-05,COUT=9.8e-05,L=5.6e-05,R1=30871,VIN=2,w1=0.000918) Y","/VIN (C1=5.7e-05,CIN=8.9e-05,COUT=1.9e-05,L=4.3e-05,R1=46163,VIN=1,w1=0.000772) Y_2","/VIN (C1=5.7e-05,CIN=8.9e-05,COUT=1.9e-05,L=4.3e-05

## 3. Processing Outputs of Dataset 1

Now we do the same for the output datasets.

In [12]:
df_output_1 = pl.read_csv(str(DATA_LOCATION + "/Output/" + "Processed_output_Vout_of_Constant_1st_Neuron_Vin0-1_v2.csv"))
df_output_2 = pl.read_csv(str(DATA_LOCATION + "/Output/" + "Processed_output_Vout_of_Constant_1st_Neuron_Vin1-2_v2.csv"))

In [13]:
df_output_3 = pl.read_csv(str(DATA_LOCATION + "/Output/" + "Processed_output_Vout_of_Constant_2nd_Neuron_Vin0-1_v2.csv"))
df_output_4 = pl.read_csv(str(DATA_LOCATION + "/Output/" + "Processed_output_Vout_of_Constant_2nd_Neuron_Vin1-2_v2.csv"))

In [14]:
%%time

df_output_1_combined = expand(df_output_1[:NUMBER_DATASET_LENGTH])
df_output_2_combined = expand(df_output_2[:NUMBER_DATASET_LENGTH])
df_output_3_combined = expand(df_output_1[:NUMBER_DATASET_LENGTH])
df_output_4_combined = expand(df_output_2[:NUMBER_DATASET_LENGTH])

CPU times: user 6.39 s, sys: 756 ms, total: 7.15 s
Wall time: 6.88 s


In [15]:
%%time
df_output_1st_neuron_dataset_1_concat = safe_concat([df_output_1_combined, df_output_2_combined])
df_output_2nd_neuron_dataset_1_concat = safe_concat([df_output_3_combined, df_output_4_combined])

CPU times: user 464 ms, sys: 15.2 ms, total: 479 ms
Wall time: 467 ms


## 4. Processing of Dataset 2

In [16]:
df_input_1 = pl.read_csv(str(DATA_LOCATION + "Input/" + "Processed_sin1_v2.csv"))
df_input_2 = pl.read_csv(str(DATA_LOCATION + "Input/" + "Processed_sin2_v2.csv"))
df_input_3 = pl.read_csv(str(DATA_LOCATION + "Input/" + "Processed_sin3_v2.csv"))

In [17]:
df_output_1 = pl.read_csv(str(DATA_LOCATION + "/Output/" + "Processed_output_Vout_of_Sin1_v2.csv"))
df_output_2 = pl.read_csv(str(DATA_LOCATION + "/Output/" + "Processed_output_Vout_of_Sin2_v2.csv"))
df_output_3 = pl.read_csv(str(DATA_LOCATION + "/Output/" + "Processed_output_Vout_of_Sin3_v2.csv"))

In [18]:
df_input_1_combined = expand(df_input_1[:NUMBER_DATASET_LENGTH])
df_input_2_combined = expand(df_input_2[:NUMBER_DATASET_LENGTH])
df_input_3_combined = expand(df_input_3[:NUMBER_DATASET_LENGTH])

df_input_dataset_2_concat = safe_concat([df_input_1_combined, df_input_2_combined, df_input_3_combined])

In [19]:
df_output_1_combined = expand(df_output_1[:NUMBER_DATASET_LENGTH])
df_output_2_combined = expand(df_output_2[:NUMBER_DATASET_LENGTH])
df_output_3_combined = expand(df_output_1[:NUMBER_DATASET_LENGTH])

df_output_dataset_2_concat = safe_concat([df_output_1_combined, df_output_2_combined, df_output_3_combined])

## 5. Combining the Datasets into One

In [20]:
# We drop the index and tranform it into a pytorch tensor 

VIN = torch.cat([
    torch.tensor(df_input_dataset_1_concat.drop("X", "X_2").to_numpy().astype("float64")),
    torch.tensor(df_output_1st_neuron_dataset_1_concat.drop("X", "X_2").to_numpy().astype("float64")),
    torch.tensor(df_input_dataset_2_concat.drop("X", "X_2", "X_3").to_numpy().astype("float64"))
], 1)

VOUT = torch.cat([
    torch.tensor(df_output_1st_neuron_dataset_1_concat.drop("X", "X_2").to_numpy().astype("float64")),
    torch.tensor(df_output_2nd_neuron_dataset_1_concat.drop("X", "X_2").to_numpy().astype("float64")),
    torch.tensor(df_output_dataset_2_concat.drop("X", "X_2", "X_3").to_numpy().astype("float64"))
], 1)

VIN.shape, VOUT.shape

(torch.Size([1507, 63993]), torch.Size([1507, 63993]))

In [21]:
X = torch.tensor(VIN.clone().detach()).float().T.unsqueeze(2)
Y = torch.tensor(VOUT.clone().detach()).float().T

X.shape, Y.shape

/tmp/ipykernel_17355/2295200637.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X = torch.tensor(VIN.clone().detach()).float().T.unsqueeze(2)
/tmp/ipykernel_17355/2295200637.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  Y = torch.tensor(VOUT.clone().detach()).float().T


(torch.Size([63993, 1507, 1]), torch.Size([63993, 1507]))

- Randomly shuffle all simulation traces.
- Toss out 50% of the data.
- Use remaining 50%:
    - 45% for training
    - 2.5% for validation
    - 2.5% for testing

In [22]:
rand_idx = torch.randperm(X.shape[0])
toss_idx = rand_idx[:int(X.shape[0]*0.5)]
train_idx = rand_idx[int(X.shape[0]*0.5):int(X.shape[0]*0.95)]
valid_idx = rand_idx[int(X.shape[0]*0.95):int(X.shape[0]*0.975)]
test_idx  = rand_idx[int(X.shape[0]*0.975):]

X_train = X[train_idx]
Y_train = Y[train_idx]
X_valid = X[valid_idx]
Y_valid = Y[valid_idx]
X_test  = X[test_idx]
Y_test  = Y[test_idx]

data = {'X_train':X_train, 'Y_train':Y_train,
        'X_valid':X_valid, 'Y_valid':Y_valid,
        'X_test':X_test, 'Y_test':Y_test}

- Normalization + downsampling
- This is a form of dimensionality reduction so the model doesn’t have to process the full 1507-point signal

In [23]:
# :1500:15 means: take columns 0, 15, 30, …, 1495.
# That gives you 100 evenly spaced points across the whole 1500-step window

for k,v in data.items():
    norms = [1.0e-04, 1.0e-04, 1.0e-04, 1.0e-04, 50000, 0.001]
    if "X" in k:
        X1 = v[:,:1500:15,:]
        X2 = v[:,1501:,:]
        for i in range(6):
            X2[:,i,:] /= norms[i]
        X = torch.cat((X1, X2), dim=1)
    else:
        X = v[:,:1500:15]

    data[k] = X

In [24]:
# Here you zoom into just the first 300 steps, but with higher resolution.
# You keep every 3rd point → again 100 points total.
# This emphasizes the early-time behavior

for k,v in data.items():
    norms = [1.0e-04, 1.0e-04, 1.0e-04, 1.0e-04, 50000, 0.001]
    if "X" in k:
        X1 = v[:,:300:3,:]
        X2 = v[:,1501:,:]
        for i in range(6):
            X2[:,i,:] /= norms[i]
        X = torch.cat((X1, X2), dim=1)
    else:
        X = v[:,:300:3]

    data[k] = X

IndexError: index 0 is out of bounds for dimension 1 with size 0

In [ ]:
# Simply take the first 100 time steps in full resolution

for k,v in data.items():
    norms = [1.0e-04, 1.0e-04, 1.0e-04, 1.0e-04, 50000, 0.001]
    if "X" in k:
        X1 = v[:,:100,:]
        X2 = v[:,1501:,:]
        for i in range(6):
            X2[:,i,:] /= norms[i]
        X = torch.cat((X1, X2), dim=1)
    else:
        X = v[:,:100]

    data[k] = X

In [24]:
# Check the size for our datasets.
for name, subset in data.items():
    print(name, subset.shape)

X_train torch.Size([28797, 106, 1])
Y_train torch.Size([28797, 100])
X_valid torch.Size([1600, 106, 1])
Y_valid torch.Size([1600, 100])
X_test torch.Size([1600, 106, 1])
Y_test torch.Size([1600, 100])


In [25]:
# Save dataset
torch.save(data, f'./data/dataset_small_v2.ds')